In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor,XGBClassifier
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df=pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/Dataset/train.csv")
train_df["filename"]="/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/train/"+train_df["filename"]

test_df=pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/Dataset/test.csv")

sub=pd.DataFrame(test_df["filename"])
sub["label"]=0

test_df["filename"]="/kaggle/input/shl-intern-hiring-assessment/Dataset/audios/test/"+test_df["filename"]

In [3]:
# import librosa

# # Sample DataFrame

# # Functions to get size and duration
# def get_size_mb(path):
#     return os.path.getsize(path) / (1024 * 1024)

# def get_duration(path):
#     try:
#         duration = librosa.get_duration(path=path)
#     except Exception as e:
#         duration = None
#     return duration

# # Apply to DataFrame
# df['size_mb'] = df['filename'].apply(get_size_mb)
# df['duration_sec'] = df['filename'].apply(get_duration)


In [4]:
import pandas as pd
import torch
import torchaudio
from tqdm import tqdm
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2Model, HubertModel

# Load DataFrame (example for train)
# Replace with actual path to train CSV
# train_df = pd.read_csv("path_to_train.csv")
# Example structure: train_df = pd.DataFrame({"filename": ["file1.wav", "file2.wav", ...]})

# Choose model: Wav2Vec2 or HuBERT
MODEL_NAME = "facebook/wav2vec2-base"  # alternative: "facebook/hubert-base-ls960"

# Load processor and model
processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
if "hubert" in MODEL_NAME:
    model = HubertModel.from_pretrained(MODEL_NAME)
else:
    model = Wav2Vec2Model.from_pretrained(MODEL_NAME)

model.eval().cuda()  # Use GPU if available; remove .cuda() for CPU

# Define hidden dimension
HIDDEN_DIM = 768  # for base Wav2Vec2/HuBERT

# Corrected function to extract mean-pooled embedding
def extract_embedding(path):
    try:
        waveform, sr = torchaudio.load(path)
        # Convert to mono if necessary
        if waveform.dim() == 2:
            if waveform.shape[0] == 1:
                waveform = waveform.squeeze(0)  # [num_samples]
            else:
                waveform = waveform.mean(dim=0)  # average channels to mono
        elif waveform.dim() == 1:
            pass
        else:
            raise ValueError(f"Unexpected waveform dimensions: {waveform.shape}")
        # Resample if necessary
        if sr != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
            waveform = resampler(waveform)
        # Process
        inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        with torch.no_grad():
            outputs = model(**{k: v.cuda() for k, v in inputs.items()})
        hidden_states = outputs.last_hidden_state  # (1, seq_len, hidden_dim)
        embedding = hidden_states.mean(dim=1).squeeze().cpu().numpy()
        return embedding
    except Exception as e:
        print(f"Error processing {path}: {e}")
        return np.zeros(HIDDEN_DIM)

# Process train dataset
train_embeddings = []
for path in tqdm(train_df['filename']):
    emb = extract_embedding(path)
    train_embeddings.append(emb)

# Create embedding DataFrame for train
train_embedding_df = pd.DataFrame(train_embeddings)
train_embedding_df.columns = [f"embed_{i}" for i in range(train_embedding_df.shape[1])]
train_final_df = pd.concat([train_df.reset_index(drop=True), train_embedding_df], axis=1)
train_final_df.to_csv("train_with_embeddings.csv", index=False)

# Process test dataset
# Replace with actual path to test CSV
# test_df = pd.read_csv("path_to_test.csv")
test_embeddings = []
for path in tqdm(test_df['filename']):
    emb = extract_embedding(path)
    test_embeddings.append(emb)

# Create embedding DataFrame for test
test_embedding_df = pd.DataFrame(test_embeddings)
test_embedding_df.columns = [f"embed_{i}" for i in range(test_embedding_df.shape[1])]
test_final_df = pd.concat([test_df.reset_index(drop=True), test_embedding_df], axis=1)
test_final_df.to_csv("test_with_embeddings.csv", index=False)

2025-05-02 18:43:11.136856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746211391.325630      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746211391.381217      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

  0%|          | 1/444 [00:00<06:40,  1.11it/s]

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

100%|██████████| 204/204 [01:16<00:00,  2.66it/s]


In [5]:
# Features and target
X = train_final_df.drop(["label","filename"],axis=1)
y = train_final_df['label']

test_final_df=test_final_df.drop("filename",axis=1)

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(train_df))

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y*2)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model = XGBRegressor(n_estimators=500,early_stopping_rounds=50)
    model.fit(X_train, y_train,eval_set=[(X_val,y_val)],verbose=100)
    
    preds = model.predict(X_val)
    oof_preds[val_idx] = preds
    sub.label+=model.predict(test_final_df)
    rmse = mean_squared_error(y_val, preds, squared=False)
    r_val, _ = pearsonr(y_val, preds)
    print(f"Fold {fold+1}: RMSE = {rmse:.4f}, R = {r_val:.4f}")

# Overall metrics
overall_rmse = mean_squared_error(y, oof_preds, squared=False)
overall_r, _ = pearsonr(y, oof_preds)

print(f"\nOOF RMSE: {overall_rmse:.4f}")
print(f"OOF R: {overall_r:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


[0]	validation_0-rmse:0.97132
[100]	validation_0-rmse:0.82821
[108]	validation_0-rmse:0.82821
Fold 1: RMSE = 0.8282, R = 0.6262
[0]	validation_0-rmse:0.96677
[93]	validation_0-rmse:0.83892
Fold 2: RMSE = 0.8389, R = 0.6220
[0]	validation_0-rmse:0.95408
[100]	validation_0-rmse:0.79136
[102]	validation_0-rmse:0.79136
Fold 3: RMSE = 0.7914, R = 0.6565
[0]	validation_0-rmse:0.98165
[55]	validation_0-rmse:0.83542
Fold 4: RMSE = 0.8344, R = 0.5985
[0]	validation_0-rmse:0.92986
[57]	validation_0-rmse:0.87584
Fold 5: RMSE = 0.8729, R = 0.5355

OOF RMSE: 0.8335
OOF R: 0.6079


In [6]:
sub["label"]/=5
sub.to_csv("sub.csv",index=False)
sub

,filename,label
0,audio_804.wav,2.626287
1,audio_1028.wav,3.270172
2,audio_865.wav,2.964924
3,audio_774.wav,3.066938
4,audio_1138.wav,2.833489
...,...,...
199,audio_787.wav,4.496142
200,audio_1079.wav,4.890176
201,audio_550.wav,3.645214
202,audio_641.wav,3.937224
